In [1]:
#importing libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_log_error
#from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
train_data = pd.read_csv('/content/drive/MyDrive/Machine Learning/train.csv')
test_data = pd.read_csv('/content/drive/MyDrive/Machine Learning/test.csv')
#op_data = pd.read_csv('sample_submission.csv')

In [2]:
train_data.head()

,date_time,deg_C,relative_humidity,absolute_humidity,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,target_carbon_monoxide,target_benzene,target_nitrogen_oxides
0,2010-03-10 18:00:00,13.1,46.0,0.7578,1387.2,1087.8,1056.0,1742.8,1293.4,2.5,12.0,167.7
1,2010-03-10 19:00:00,13.2,45.3,0.7255,1279.1,888.2,1197.5,1449.9,1010.9,2.1,9.9,98.9
2,2010-03-10 20:00:00,12.6,56.2,0.7502,1331.9,929.6,1060.2,1586.1,1117.0,2.2,9.2,127.1
3,2010-03-10 21:00:00,11.0,62.4,0.7867,1321.0,929.0,1102.9,1536.5,1263.2,2.2,9.7,177.2
4,2010-03-10 22:00:00,11.9,59.0,0.7888,1272.0,852.7,1180.9,1415.5,1132.2,1.5,6.4,121.8


In [3]:
test_data.head()

,date_time,deg_C,relative_humidity,absolute_humidity,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5
0,2011-01-01 00:00:00,8.0,41.3,0.4375,1108.8,745.7,797.1,880.0,1273.1
1,2011-01-01 01:00:00,5.1,51.7,0.4564,1249.5,864.9,687.9,972.8,1714.0
2,2011-01-01 02:00:00,5.8,51.5,0.4689,1102.6,878.0,693.7,941.9,1300.8
3,2011-01-01 03:00:00,5.0,52.3,0.4693,1139.7,916.2,725.6,1011.0,1283.0
4,2011-01-01 04:00:00,4.5,57.5,0.4650,1022.4,838.5,871.5,967.0,1142.3


In [4]:
#op_data.head()

In [5]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7111 entries, 0 to 7110
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   date_time               7111 non-null   object 
 1   deg_C                   7111 non-null   float64
 2   relative_humidity       7111 non-null   float64
 3   absolute_humidity       7111 non-null   float64
 4   sensor_1                7111 non-null   float64
 5   sensor_2                7111 non-null   float64
 6   sensor_3                7111 non-null   float64
 7   sensor_4                7111 non-null   float64
 8   sensor_5                7111 non-null   float64
 9   target_carbon_monoxide  7111 non-null   float64
 10  target_benzene          7111 non-null   float64
 11  target_nitrogen_oxides  7111 non-null   float64
dtypes: float64(11), object(1)
memory usage: 666.8+ KB


In [6]:
features = ['deg_C', 'relative_humidity', 'absolute_humidity', 'sensor_1', 'sensor_2', 'sensor_3', 'sensor_4', 'sensor_5']
target = ['target_carbon_monoxide', 'target_benzene', 'target_nitrogen_oxides']

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
X_train, X_val, y_train, y_val = train_test_split(train_data[features], train_data[target], test_size=0.2, random_state=42)

In [9]:
rf_carbon_monoxide = RandomForestRegressor(n_estimators=100, random_state=42)
rf_carbon_monoxide.fit(X_train, y_train['target_carbon_monoxide'])
carbon_monoxide_preds = rf_carbon_monoxide.predict(X_val)

rf_benzene = RandomForestRegressor(n_estimators=100, random_state=42)
rf_benzene.fit(X_train, y_train['target_benzene'])
benzene_preds = rf_benzene.predict(X_val)

rf_nitrogen_oxides = RandomForestRegressor(n_estimators=100, random_state=42)
rf_nitrogen_oxides.fit(X_train, y_train['target_nitrogen_oxides'])
nitrogen_oxides_preds = rf_nitrogen_oxides.predict(X_val)


In [10]:
from sklearn.metrics import mean_squared_log_error

rmsle_carbon_monoxide = np.sqrt(mean_squared_log_error(y_val['target_carbon_monoxide'], carbon_monoxide_preds))
rmsle_benzene = np.sqrt(mean_squared_log_error(y_val['target_benzene'], benzene_preds))
rmsle_nitrogen_oxides = np.sqrt(mean_squared_log_error(y_val['target_nitrogen_oxides'], nitrogen_oxides_preds))
overall= rmsle_carbon_monoxide + rmsle_benzene + rmsle_nitrogen_oxides

print('RMSLE for carbon monoxide:', rmsle_carbon_monoxide)
print('RMSLE for benzene:', rmsle_benzene)
print('RMSLE for nitrogen oxides:', rmsle_nitrogen_oxides)
print("Overall: ",overall/3)
carbon_monoxide_test_preds = rf_carbon_monoxide.predict(test_data[features])
benzene_test_preds = rf_benzene.predict(test_data[features])
nitrogen_oxides_test_preds = rf_nitrogen_oxides.predict(test_data[features])

submission = pd.DataFrame({
    'date_time': test_data['date_time'],
    'target_carbon_monoxide': carbon_monoxide_test_preds,
    'target_benzene': benzene_test_preds,
    'target_nitrogen_oxides': nitrogen_oxides_test_preds
})

submission.to_csv('submission.csv', index=False)

RMSLE for carbon monoxide: 0.1361581831870301
RMSLE for benzene: 0.09393850643737371
RMSLE for nitrogen oxides: 0.33143521875786836
Overall:  0.18717730279409073


In [11]:
!pip install -q gradio
import gradio as gr

def airpollution(deg_C,relative_humidity,absolute_humidity,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5):
    df = pd.DataFrame.from_dict({'deg_C': [deg_C], 'relative_humidity': [relative_humidity], 'absolute_humidity': [absolute_humidity],'sensor_1':[sensor_1],'sensor_2':[sensor_2],'sensor_3':[sensor_3],'sensor_4':[sensor_4],'sensor_5':[sensor_5]})
    
    pred = rf_carbon_monoxide.predict(df)[0]
    pred2 = rf_benzene.predict(df)[0]
    pred3 =rf_nitrogen_oxides.predict(df)[0]
    print(pred,pred2,pred3)
    return pred,pred2,pred3


deg_C = gr.inputs.Textbox( label="deg_C")
relative_humidity= gr.inputs.Textbox(label="relative_humidity")
absolute_humidity= gr.inputs.Textbox(label="absolute_humidity")
sensor_1= gr.inputs.Textbox(label="sensor_1")
sensor_2= gr.inputs.Textbox(label="sensor_2")
sensor_3= gr.inputs.Textbox(label="sensor_3")
sensor_4= gr.inputs.Textbox(label="sensor_4")
sensor_5= gr.inputs.Textbox(label="sensor_5")
# CarbonMonoxide= gr.outputs.Textbox(label="CarbonMonoxide")
# Benzene = gr.outputs.Textbox(label="Benzene")
# NitrogenOxide = gr.outputs.Textbox(label="NitrogenOxide")
output = [gr.outputs.Textbox(label="CarbonMonoxide"),gr.outputs.Textbox(label="Benzene"), gr.outputs.Textbox(label="NitrogenOxide")]
gr.Interface(airpollution,[deg_C,relative_humidity,absolute_humidity,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5],output).launch();



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 KB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 KB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.1/144.1 KB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 KB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 KB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 KB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 KB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 24.8 MB/s eta 0:

/usr/local/lib/python3.9/dist-packages/gradio/inputs.py:27: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.9/dist-packages/gradio/deprecation.py:40: UserWarning: `numeric` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.9/dist-packages/gradio/outputs.py:22: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>